In [1]:
import json

import pandas as pd
import pytorch_lightning as pl
import timm
import torch
from PIL import Image
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm

D:\ganka_2024\jsaio_2024_solution\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DEBUG = False

In [3]:
CONFIG_JSON_PATH = "../config/config.json"

with open(CONFIG_JSON_PATH, encoding="utf-8") as f:
    cfg = json.load(f)

SUBMISSION_PATH = cfg["dataset"]["submission"]
TEST_CROP_DATASET_DIR = cfg["dataset"]["test_crop"]

BATCH_SIZE = cfg["params"]["batch_size"]
NUM_WORKERS = cfg["params"]["num_workers"]
IMG_SIZE = cfg["params"]["img_size"]
TEST_SIZE = cfg["params"]["test_size"]
SEED = cfg["params"]["seed"]

MAX_EPOCH = 20

In [4]:
cfg

{'dataset': {'label_train': 'D:\\ganka_2024\\dataset\\label_train.csv',
  'submission': 'D:\\ganka_2024\\dataset\\解答用ファイル.csv',
  'train': 'D:\\ganka_2024\\dataset\\train',
  'train_crop': 'D:\\ganka_2024\\dataset\\train_crop',
  'test': 'D:\\ganka_2024\\dataset\\test',
  'test_crop': 'D:\\ganka_2024\\dataset\\test_crop'},
 'params': {'model_name': 'tf_efficientnetv2_s.in21k_ft_in1k',
  'batch_size': 32,
  'num_workers': 0,
  'img_size': 224,
  'test_size': 0.2,
  'seed': 1024,
  'learning_rate': 0.001}}

In [5]:
pl.seed_everything(SEED, workers=True)

Seed set to 1024


1024

In [6]:
df = pd.read_csv(SUBMISSION_PATH)

### dataset

In [7]:
class PupilTestCSV(Dataset):
    def __init__(self, df: pd.DataFrame, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.loc[idx, 'path']

        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image

### Dataloader

### model

In [8]:
class PupilModel(pl.LightningModule):
    def __init__(
            self,
            model_name: str = "maxvit_tiny_tf_224.in1k",
            pretrained: bool = True,
            num_classes: int = 2,
            learning_rate: float = 1e-3,
    ):
        super().__init__()
        self.save_hyperparameters()

        self.model = timm.create_model(
            model_name,
            pretrained=pretrained,
            num_classes=num_classes
        )

        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

### inference

In [9]:
import os

df = pd.read_csv(SUBMISSION_PATH)

if DEBUG:
    df = df.head(300)

df['path'] = df['File'].apply(lambda x: os.path.join(TEST_CROP_DATASET_DIR, x))

### Dataloader

In [10]:
test_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

test_dataset = PupilTestCSV(
    df,
    transform=test_transforms,
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

### load model

In [11]:
base_path = "../2_train_seed_ensemble"
experiments_folds = [("exp040", [1024, 1025, 1026, 1027, 1028], [0, 1, 2, 3, 4]),
                     ("exp042", [2024, 2025, 2026, 2027, 2028], [0, 1, 2, 3, 4])]

ckpt_files = []

for exp, seeds, folds in experiments_folds:
    for seed in seeds:
        for fold in folds:
            ckpt_files.append(os.path.join(base_path, exp, "checkpoints", f"final_model_seed_{seed}_fold_{fold}.pth"))

ckpt_files

['../2_train_seed_ensemble\\exp040\\checkpoints\\final_model_seed_1024_fold_0.pth',
 '../2_train_seed_ensemble\\exp040\\checkpoints\\final_model_seed_1024_fold_1.pth',
 '../2_train_seed_ensemble\\exp040\\checkpoints\\final_model_seed_1024_fold_2.pth',
 '../2_train_seed_ensemble\\exp040\\checkpoints\\final_model_seed_1024_fold_3.pth',
 '../2_train_seed_ensemble\\exp040\\checkpoints\\final_model_seed_1024_fold_4.pth',
 '../2_train_seed_ensemble\\exp040\\checkpoints\\final_model_seed_1025_fold_0.pth',
 '../2_train_seed_ensemble\\exp040\\checkpoints\\final_model_seed_1025_fold_1.pth',
 '../2_train_seed_ensemble\\exp040\\checkpoints\\final_model_seed_1025_fold_2.pth',
 '../2_train_seed_ensemble\\exp040\\checkpoints\\final_model_seed_1025_fold_3.pth',
 '../2_train_seed_ensemble\\exp040\\checkpoints\\final_model_seed_1025_fold_4.pth',
 '../2_train_seed_ensemble\\exp040\\checkpoints\\final_model_seed_1026_fold_0.pth',
 '../2_train_seed_ensemble\\exp040\\checkpoints\\final_model_seed_1026_fold_

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_models = []

for ckpt_file in ckpt_files:
    model = PupilModel.load_from_checkpoint(ckpt_file).to(device)
    model.eval()
    model.freeze()
    best_models.append(model)

### inference

In [13]:
transforms = [
    lambda x: x,  # 元の入力
    lambda x: torch.flip(x, dims=[-1]),  # 水平フリップ
    lambda x: torch.flip(x, dims=[-2]),  # 垂直フリップ
]

all_predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        inputs = batch
        inputs = inputs.to(device)

        batch_outputs = []
        for model in best_models:
            model_outputs = []
            for transform in transforms:
                transformed_inputs = transform(inputs)
                outputs = model(transformed_inputs)
                outputs, _ = torch.split(outputs, [2, 7], dim=1)  # 非メタボの確率 / メタボの確率 のみ取得
                model_outputs.append(outputs)
            batch_outputs.extend(model_outputs)

        outputs = torch.stack(batch_outputs, dim=0)

        outputs_mean = outputs.mean(dim=0)

        _, preds = torch.max(outputs_mean, dim=1)

        all_predictions.extend(preds.cpu().numpy())

  0%|          | 0/16 [00:00<?, ?it/s]D:\ganka_2024\jsaio_2024_solution\venv\lib\site-packages\timm\models\maxxvit.py:260: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  x = torch.nn.functional.scaled_dot_product_attention(
100%|██████████| 16/16 [01:14<00:00,  4.67s/it]


In [14]:
df["prediction"] = all_predictions
submission_df = df[['File', 'prediction']]
submission_df = submission_df.rename(columns={'prediction': 'MetabolicSyndrome_0=No_1=Yes'})

In [15]:
submission_df.to_csv("解答用ファイル.csv", index=False)